### Data preparation

#### Import packages

In [181]:
pip install pulp

Note: you may need to restart the kernel to use updated packages.


In [182]:
# Install the needed libraries
import pandas as pd
import numpy as np
from pulp import *
import time
import warnings
from pandas.core.common import SettingWithCopyWarning

In [183]:
# Ignore SettingWithCopyWarning 
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

#### Import data

In [184]:
# Import the data
data = pd.read_excel('data_python_modified.xlsx', sheet_name='data', index_col=0)
real = pd.read_excel('data_python_modified.xlsx', sheet_name='реал', index_col=0)
bandwidth = pd.read_excel('data_python_modified.xlsx', sheet_name='ПС', index_col=0)
match = pd.read_excel('data_python_modified.xlsx', sheet_name='match', index_col=0)

#### Rename variables

In [185]:
# Create a dict of names on a spreadsheets that we are going to use
names = {"Тариф ж/д" : "t1",
        "Тариф хранение" : "t2",
        "Тариф бренд" : "t3",
        "Тариф ВЛ" : "t4",
        "Плечо, км" : "distance",
        "Дата" : "date",
        "НБ" : "origin",
        "ОУ" : "point",
        "НГ" : "brand",
        "НГ_ПС" : "product",
        "Объем" : "volume"} 

In [186]:
# Rename the names in dataframes
data.rename(columns=names, inplace=True)
real.rename(columns=names, inplace=True)
bandwidth.rename(columns=names, inplace=True)
match.rename(columns=names, inplace=True)

#### Reorganize data

In [187]:
# Function for calculating the tarif
def calculate_tarif(row):
    if row["distance"] <= 50:
        return row["t1"] + row["t2"] + row["t3"] + row["t4"]
    else:
        return row["t1"] + row["t2"] + row["t3"] +  row["t4"] * row["distance"]

In [188]:
# Calculate cost
data['cost'] = data.apply(lambda row: calculate_tarif(row), axis=1)

In [189]:
# Checking dates
dates = pd.DataFrame({"date" : data['date'].unique()})
dates

,date
0,2023-04-01
1,2023-03-01


In [190]:
# Save the first date to the t 
t = dates.iloc[0, 0]

In [191]:
# Subset for the time period
data = data[data['date'] == t]
real = real[real['date'] == t]
bandwidth = bandwidth[bandwidth['date'] == t]
    
# Adjust datatypes
data['origin'] = data['origin'].map(lambda x: int(x.strip('Нефтебаза ')))
data['point'] = data['point'].map(lambda x: int(x.strip('АЗС ')))
real['point'] = real['point'].map(lambda x: int(x.strip('АЗС ')))
bandwidth['origin'] = bandwidth['origin'].map(lambda x: int(x.strip('Нефтебаза ')))

In [192]:
# Create some dictionaries for later usage
match_brand_product = dict(zip(match['brand'], match['product']))
match_brand_number = dict(enumerate(match['brand']))
match_product_number = dict(enumerate(match['product']))

match_product_brand = dict()
for key, value in match_brand_product.items():
    match_product_brand.setdefault(value, list()).append(key)
    
match_number_brand = dict()
for key, value in match_brand_number.items():
    match_number_brand[value] = key
    
match_number_product = dict()
for key, value in match_product_number.items():
    match_number_product[value] = key

In [193]:
# Store origin with infinite supply in a separate list
origins_with_inf_supply = list(bandwidth[bandwidth['volume'] == np.inf]['origin'].unique())

In [194]:
# Function for calculating the tarif
def create_var_Name(row):
    return 'P' + str(int(row["point"])) + 'O' + str(int(row["origin"])) +  '_brand_' + str(int(row["brand_number"]))

In [195]:
# Reorganize the data
columns_to_keep = ['origin', 'point', 'brand', 'product', 'cost']

df = data[columns_to_keep].reset_index().drop('index', axis=1).reset_index()

# Code brand, product
df['brand_number'] = df['brand'].map(lambda x: match_number_brand.get(x))
df['product_number'] = df['product'].map(lambda x: match_number_product.get(x))

# Create a name variable
df['var_Name'] = df.apply(lambda row: create_var_Name(row), axis=1) 

In [196]:
df.head()

,index,origin,point,brand,product,cost,brand_number,product_number,var_Name
0,0,0,0,Бензин 100 бренд,АИ 100,3268.488947,0,0,P0O0_brand_0
1,1,0,0,Бензин 92,Аи 92,3268.488947,1,1,P0O0_brand_1
2,2,0,0,Бензин 95,Аи 95,3268.488947,2,3,P0O0_brand_2
3,3,0,0,Бензин 95 бренд,Аи 95,3338.651597,3,3,P0O0_brand_3
4,4,0,0,Топливо дизельное с присадками летнее,ДТ,3133.526439,4,4,P0O0_brand_4


In [197]:
# Create demand df
demand = real[['point', 'brand', 'product', 'volume']]
demand.rename(columns={'volume':'demand'}, inplace=True)
demand['brand_number'] = demand['brand'].map(lambda x: match_number_brand.get(x))
demand['product_number'] = demand['product'].map(lambda x: match_number_product.get(x))

In [198]:
demand.head()

,point,brand,product,demand,brand_number,product_number
0,0,Бензин 100 бренд,АИ 100,29.693803,0,0
1,0,Бензин 92,Аи 92,304.570984,1,1
2,0,Бензин 95,Аи 95,281.706487,2,3
3,0,Бензин 95 бренд,Аи 95,65.738768,3,3
4,0,Топливо дизельное с присадками летнее,ДТ,71.355788,4,4


In [199]:
# Create supply df
supply = bandwidth[['origin', 'product', 'volume']]
supply.rename(columns={'volume':'supply'}, inplace=True)
supply['product_number'] = supply['product'].map(lambda x: match_number_product.get(x))

In [200]:
supply.head()

,origin,product,supply,product_number
4,0,Аи 92,4663.7,1
5,0,Аи 95,5078.7,3
6,0,АИ 100,331.1,0
7,0,ДТ,4881.1,4
12,1,Аи 92,inf,1


### Usage of puLP

In [201]:
# del problem

#### Define function

In [202]:
# Set function's goal
problem = pulp.LpProblem('0', sense=LpMinimize)

In [203]:
# Create continuous variables 
x = [pulp.LpVariable(df['var_Name'][i], lowBound = 0, cat='Continuous') for i in np.arange(0, len(df))]

In [204]:
# Create a function
problem += pulp.LpAffineExpression([(x[i], df['cost'][i]) for i in np.arange(0, len(df))])

#### Define restriction #1: demand equals const

In [205]:
n_constraint = 1
brands = df['brand_number'].unique()

In [206]:
for b in brands:
    df1 = df[df['brand_number'] == b]
    points = df1['point'].unique()
    
    for p in points:
        df2 = df1[df1['point'] == p]
        const = float(demand[(demand['point'] == p) & (demand['brand_number'] == b)]['demand'])
        # Add a restriction on demand == const
        problem+= pulp.LpAffineExpression([(x[i], 1) for i in df2.index ]) == const, str(n_constraint)
        n_constraint +=1

#### Define restriction #2: supply less or equal than const;  restriction #4: supply greater or equal to const * percent_loading

In [207]:
# Define percents of loading demanded by management
percents_loading = np.arange(0.5, 1, 0.1)
percent_loading = percents_loading[0]

In [208]:
# Function for dividing supply between different brands
def merge_supply(brand_number, match_brand_product, match_brand_number, match_product_brand, match_number_brand):
    """This function returns a set of brands' numbers that are produced from the same product"""
    brand_name = match_brand_number[brand_number]
    product_name = match_brand_product[brand_name]
    brands_overlap_names = match_product_brand[product_name]
    brands_overlap_numbers = set(match_number_brand.get(item) for item in brands_overlap_names)

    return brands_overlap_numbers

In [209]:
# Set for storing already processed brands 
brands_accounted_for = set()

In [210]:
for b in brands:
    
    if b in brands_accounted_for:
        continue
    else:
        brands_overlap_numbers = merge_supply(b, match_brand_product, match_brand_number, match_product_brand, match_number_brand)
        brands_accounted_for.update(brands_overlap_numbers)
        
        df1 = df[df['brand_number'] == b]
        origins = df1['origin'].unique()
        
        for o in origins:
            
            if o in origins_with_inf_supply:
                continue
            else:
                
                df2 = df1[df1['origin'] == o]
                product_name = match_brand_product[match_brand_number[b]]
                const = float(supply[(supply['origin'] == o) & (supply['product'] == product_name)]['supply'])
                
                # Add a restriction on supply <= upper_const
                problem+= pulp.LpAffineExpression([(x[i], 1) for i in df2.index ]) <= const, str(n_constraint)
                n_constraint +=1

                # Add a restriction on supply >= lower_const
                const *= percent_loading
                problem+= pulp.LpAffineExpression([(x[i], 1) for i in df2.index ]) >= const, str(n_constraint)
                n_constraint +=1        

#### Define restriction #3: all brands for one point should come from the same origin

In [211]:
M = 10**9
dummies = {}

In [212]:
# Function for creating dummies names
def create_dummy_Name(row):
    return 'y_P' + str(int(row["point"])) + 'O' + str(int(row["origin"]))

In [213]:
# Create a name variable
dummies_names = set(df.apply(lambda row: create_dummy_Name(row), axis=1))

# Create dummy variables and store them in a dict
for name in dummies_names:
    dummies[name] = pulp.LpVariable(name, lowBound=0, upBound = 1, cat = 'Integer')

In [214]:
points = df['point'].unique()
origins = df['origin'].unique()

for p in points:
    for o in origins:
        df1 = df[(df['point'] == p) & (df['origin'] == o)]
        
        string = "y_P" + str(p) + "O" + str(o)
        d = dummies.get(string)
        
        problem+= pulp.LpAffineExpression([(x[i], 1) for i in df1.index ]) <=  M * d, str(n_constraint)
        n_constraint += 1

In [215]:
for p in points:
    trans = []
    
    for o in origins:
        string = "y_P" + str(p) + "O" + str(o)
        trans.append(dummies.get(string))
        
    summa = sum(np.array(trans))
    
    # Add a restriction on dummies
    problem += summa == 1, str(n_constraint)
    n_constraint += 1

In [216]:
problem

0:
MINIMIZE
3268.48894708041*P0O0_brand_0 + 3268.48894708041*P0O0_brand_1 + 3268.48894708041*P0O0_brand_2 + 3338.6515970804103*P0O0_brand_3 + 3133.5264394962396*P0O0_brand_4 + 4808.4651*P0O1_brand_0 + 4808.4651*P0O1_brand_1 + 4808.4651*P0O1_brand_2 + 4854.0051*P0O1_brand_3 + 4734.995826408034*P0O1_brand_4 + 3217.86602068041*P10O0_brand_0 + 3217.86602068041*P10O0_brand_1 + 3217.86602068041*P10O0_brand_2 + 3288.0286706804104*P10O0_brand_3 + 3082.9035130962393*P10O0_brand_4 + 4808.4651*P10O1_brand_0 + 4808.4651*P10O1_brand_1 + 4808.4651*P10O1_brand_2 + 4854.0051*P10O1_brand_3 + 4734.995826408034*P10O1_brand_4 + 3237.0776492804102*P11O0_brand_0 + 3237.0776492804102*P11O0_brand_1 + 3237.0776492804102*P11O0_brand_2 + 3307.2402992804105*P11O0_brand_3 + 3102.1151416962393*P11O0_brand_4 + 4808.4651*P11O1_brand_0 + 4808.4651*P11O1_brand_1 + 4808.4651*P11O1_brand_2 + 4854.0051*P11O1_brand_3 + 4734.995826408034*P11O1_brand_4 + 3238.0467818804104*P12O0_brand_0 + 3238.0467818804104*P12O0_brand_1 + 3

#### Solve the problem and print the results

In [217]:
start = time.time()

In [218]:
problem.solve()

1

In [219]:
# Print the results
print ("Результат:")
for variable in problem.variables():
    print (variable.name, "=", variable.varValue)
print ("Стоимость доставки:")
print (abs(problem.objective.value()))
stop = time.time()
print ("Время :")
print(stop - start)

Результат:
P0O0_brand_0 = 29.693803
P0O0_brand_1 = 304.57098
P0O0_brand_2 = 281.70649
P0O0_brand_3 = 65.738768
P0O0_brand_4 = 71.355788
P0O1_brand_0 = 0.0
P0O1_brand_1 = 0.0
P0O1_brand_2 = 0.0
P0O1_brand_3 = 0.0
P0O1_brand_4 = 0.0
P10O0_brand_0 = 42.384663
P10O0_brand_1 = 268.45006
P10O0_brand_2 = 276.05923
P10O0_brand_3 = 67.783332
P10O0_brand_4 = 108.14143
P10O1_brand_0 = 0.0
P10O1_brand_1 = 0.0
P10O1_brand_2 = 0.0
P10O1_brand_3 = 0.0
P10O1_brand_4 = 0.0
P11O0_brand_0 = 18.717185
P11O0_brand_1 = 182.12777
P11O0_brand_2 = 175.19265
P11O0_brand_3 = 40.864389
P11O0_brand_4 = 110.71506
P11O1_brand_0 = 0.0
P11O1_brand_1 = 0.0
P11O1_brand_2 = 0.0
P11O1_brand_3 = 0.0
P11O1_brand_4 = 0.0
P12O0_brand_0 = 29.952396
P12O0_brand_1 = 241.90937
P12O0_brand_2 = 246.86835
P12O0_brand_3 = 54.433692
P12O0_brand_4 = 46.746832
P12O1_brand_0 = 0.0
P12O1_brand_1 = 0.0
P12O1_brand_2 = 0.0
P12O1_brand_3 = 0.0
P12O1_brand_4 = 0.0
P13O0_brand_1 = 150.43719
P13O0_brand_2 = 68.076836
P13O0_brand_3 = 38.306174
P

In [ ]:
cvxopt - 1

In [ ]:
pulp - solver CBC default 

In [ ]:
scipy.optimize - матрица

In [ ]:
# cvxpty - 